In [ ]:
import numpy as np
import pandas as pd 
import seaborn as sbs
import matplotlib.pyplot as plt 
%matplotlib inline
import warnings
import requests
import dataframe_image as dfi
warnings.filterwarnings('ignore')

In [ ]:
qmc_data = pd.read_excel('Data.xlsx')

In [ ]:
qmc_data.head()

In [ ]:
qmc_data.info()

In [ ]:
data['Avg_Monthly_Trips'] = qmc_data['Avg_Monthly_Trips'].round(decimals=2)
qmc_data['Medicare_Percent'] = qmc_data['Medicare_Percent'].round(decimals=2)
qmc_data['Medicaid_Percent'] = qmc_data['Medicaid_Percent'].round(decimals =2)
qmc_data['Insurance_Percent'] = qmc_data['Insurance_Percent'].round(decimals =2)
qmc_data['Contract_Percent'] = qmc_data['Contract_Percent'].round(decimals =2)
qmc_data['Bill_Patient_Percent'] = qmc_data['Bill_Patient_Percent'].round(decimals =2)
qmc_data['Private_Pay_Percent'] = qmc_data['Private_Pay_Percent'].round(decimals =2)
qmc_data['Emerg_Percent_Trips'] = qmc_data['Emerg_Percent_Trips'].round(decimals =2)
qmc_data['Non_Emerg_Percent_Trip'] = qmc_data['Non_Emerg_Percent_Trip'].round(decimals =2)
qmc_data['Avg_Rev_Per_Trip'] = qmc_data['Avg_Rev_Per_Trip'].round(decimals =2)
qmc_data['Avg_Cogs_Per_Trip'] = qmc_data['Avg_Cogs_Per_Trip'].round(decimals =2)
qmc_data['Avg_Profit_Per_Trip'] = qmc_data['Avg_Profit_Per_Trip'].round(decimals =2)

In [ ]:
qmc_data.head()

In [ ]:
service_df = pd.read_excel('service_type.xlsx')
service_df.head()

In [ ]:
qmc_data.rename(columns={'CoName':'name'}, inplace=True)

In [ ]:
qmc_data = pd.merge(qmc_data, service_df[['name', 'Market Type','Ground/Air']], on=['name'], how='left')
qmc_data.head()

In [ ]:
qmc_data.info()

In [ ]:
qmc_data = qmc_data.dropna()

In [ ]:
### importing population data for each city using census api
key='885f59de6382eda46280415daba964af9a0fc190'
year = '2019'
dsource = 'pep'
dname ='population'
base_url = f'https://api.census.gov/data/{year}/{dsource}/{dname}?get=NAME,POP&for=place:*&key={key}'
response= requests.get(base_url)

In [ ]:
print(response.text)

In [ ]:
pop_data = response.json()

In [ ]:
pop_df = pd.DataFrame(pop_data[1:])
pop_df.head()

In [ ]:
pop_df= pop_df.drop([2,3], axis=1)
pop_df.rename(columns={0:'city', 1:'Population'}, inplace=True)
pop_df.head()

In [ ]:
pop_df[['city_name','state']] = pop_df.city.str.split( ',', expand=True)
pop_df.head()

In [ ]:
pop_df= pop_df.drop(['city'], axis=1)
pop_df.head()

In [ ]:
pop_df['city_name'] = pop_df['city_name'].str.replace(r'city$','')
pop_df['city_name'] = pop_df['city_name'].str.replace(r'town$','')

In [ ]:
pop_df.rename(columns={'city_name':'city'}, inplace=True)

In [ ]:
pop_df['state']= pop_df['state'].str.strip()
pop_df['city']= pop_df['city'].str.strip()

In [ ]:
us_state_abbrev = {'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO',
'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA',
'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'}

In [ ]:
pop_df['state'] = pop_df['state'].map(us_state_abbrev).fillna(pop_df['state'])
pop_df.head()

In [ ]:
pop_df['city'] = pop_df['city'].str.upper()

In [ ]:
qmc_data = pd.merge(qmc_data,pop_df[['Population','city','state']], on=['city','state'], how='left')
qmc_data.head()

In [ ]:
qmc_data.info()

In [ ]:
qmc_data.to_csv(r'Data_File.csv', index = False)

In [ ]:
qmc_data = qmc_data.drop(['Population'], axis=1)

In [ ]:
qmc_data['Market Type']= qmc_data['Market Type'].str.strip()
qmc_data['Ground/Air']= qmc_data['Ground/Air'].str.strip()

### Explantory Data Analysis

In [ ]:
qmc_data.describe()

In [ ]:
sbs.pairplot(qmc_data)

In [ ]:
df_state_total_trips = qmc_data.pivot_table(values = 'Trip_Count', index = 'state', aggfunc = 'sum')
df_state_total_trips.reset_index(inplace=True)
df_state_total_trips.sort_values(by=('Trip_Count'))

In [ ]:
plt.figure(figsize=(13,7))
plt.xticks(rotation=45)
plt.title("Total Trips By State", fontsize=16,color="black")
sbs.barplot(df_state_total_trips["state"],y= df_state_total_trips["Trip_Count"])
plt.savefig('Trips Total By State')

In [ ]:
df_medicaid = qmc_data.pivot_table(values = 'Medicaid_Percent', index = 'state', aggfunc = 'mean')
df_medicaid.reset_index(inplace=True)
df_medicaid['Medicaid_Percent'] = df_medicaid['Medicaid_Percent'].round(decimals=2)
df_medicaid.sort_values(by='Medicaid_Percent')

In [ ]:
plt.figure(figsize=(13,7))
plt.xticks(rotation=45)
plt.title("Average Percent of Medicaid Trip by State", fontsize=16,color="black")
sbs.barplot(df_medicaid["state"],y= df_medicaid["Medicaid_Percent"])
plt.savefig('Average Medicaid Percent')

In [ ]:
df_medicare = qmc_data.pivot_table(values = 'Medicare_Percent', index = 'state', aggfunc = 'mean')
df_medicare.reset_index(inplace=True)
df_medicare['Medicare_Percent'] = df_medicare['Medicare_Percent'].round(decimals=2)
df_medicare.sort_values(by='Medicare_Percent')

In [ ]:
plt.figure(figsize=(13,7))
plt.xticks(rotation=45)
plt.title("Average Percent of Medicare Trip by State", fontsize=16,color="black")
sbs.barplot(df_medicare["state"],y= df_medicare["Medicare_Percent"])
plt.savefig('Average Medicare Percent')

In [ ]:
df_insurance = qmc_data.pivot_table(values = 'Insurance_Percent', index = 'state', aggfunc = 'mean')
df_insurance.reset_index(inplace=True)
df_insurance['Insurance_Percent'] = df_insurance['Insurance_Percent'].round(decimals=2)
df_insurance.sort_values(by='Insurance_Percent')

In [ ]:
plt.figure(figsize=(13,7))
plt.xticks(rotation=45)
plt.title("Average Percent of Insurance Trip by State", fontsize=16,color="black")
sbs.barplot(df_insurance["state"],y= df_insurance["Insurance_Percent"])
plt.savefig('Average Insurance Percent')

In [ ]:
df_bill_payment = qmc_data.pivot_table(values = 'Bill_Patient_Percent', index = 'state', aggfunc = 'mean')
df_bill_payment.reset_index(inplace=True)
df_bill_payment['Bill_Patient_Percent'] = df_bill_payment['Bill_Patient_Percent'].round(decimals=2)
df_bill_payment.sort_values(by='Bill_Patient_Percent')

In [ ]:
plt.figure(figsize=(13,7))
plt.xticks(rotation=45)
plt.title("Average Percent of Bill Payment Trip by State", fontsize=16,color="black")
sbs.barplot(df_bill_payment["state"],y= df_bill_payment["Bill_Patient_Percent"])
plt.savefig('Bill_Patient_Percent')

In [ ]:
df_contract = qmc_data.pivot_table(values = 'Contract_Percent', index = 'state', aggfunc = 'mean')
df_contract.reset_index(inplace=True)
df_contract['Contract_Percent'] = df_contract['Contract_Percent'].round(decimals=2)
df_contract.sort_values(by='Contract_Percent')

In [ ]:
plt.figure(figsize=(13,7))
plt.xticks(rotation=45)
plt.title("Average Percent of Contract Trip by State", fontsize=16,color="black")
sbs.barplot(df_contract["state"],y= df_contract["Contract_Percent"])
plt.savefig('Contract_Percent')

In [ ]:
df_cogs = qmc_data.pivot_table(values = 'TripCOGS', index = 'state', aggfunc = 'mean')
df_cogs.reset_index(inplace=True)
df_cogs['TripCOGS'] = df_cogs['TripCOGS'].round(decimals=2)
df_cogs.sort_values(by='TripCOGS')

In [ ]:
plt.figure(figsize=(13,7))
plt.xticks(rotation=45)
plt.ticklabel_format(style='plain')
plt.title("Average COGS By State", fontsize=16,color="black")
sbs.barplot(df_cogs["state"],y= df_cogs["TripCOGS"])
plt.savefig('Average_Cogs_By_State.jpg')

In [ ]:
avg_profit = qmc_data.pivot_table(values ='Profit', index = 'state', aggfunc = 'mean')
avg_profit.reset_index(inplace=True)
avg_profit['Profit'] = avg_profit['Profit'].round(decimals=2)
avg_profit.sort_values(by='Profit')

In [ ]:
plt.figure(figsize=(13,7))
plt.xticks(rotation=45)
plt.ticklabel_format(style='plain')
plt.title("Average Profit By State", fontsize=16,color="black")
sbs.barplot(avg_profit["state"],y= avg_profit["Profit"])
plt.savefig('Average_Profit_By_State.jpg')

### Encoding of Categorical Data

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [ ]:
df = qmc_data.drop(['name'],axis=1)

In [ ]:
df['city'] = encoder.fit_transform(df['city'])
df['state'] = encoder.fit_transform(df['state'])
df['Geo'] = encoder.fit_transform(df['Geo'])
df['Market Type'] = encoder.fit_transform(df['Market Type'])
df['Ground/Air'] = encoder.fit_transform(df['Ground/Air'])

### Customer Segmentation Model 1 All Features

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
test= df['city'].values #convert to numpy array
y = scaler.fit_transform(test.reshape(-1,1))
df['city'] = y
test= df['state'].values #convert to numpy array
y = scaler.fit_transform(test.reshape(-1,1))
df['state'] = y
test= df['Geo'].values #convert to numpy array
y = scaler.fit_transform(test.reshape(-1,1))
df['Geo'] = y
test= df['Months_Customer'].values #convert to numpy array
y = scaler.fit_transform(test.reshape(-1,1))
df['Months_Customer'] = y
test= df['Avg_Monthly_Trips'].values #convert to numpy array
y = scaler.fit_transform(test.reshape(-1,1))
df['Avg_Monthly_Trips'] = y
test= df['Trip_Count'].values #convert to numpy array
y = scaler.fit_transform(test.reshape(-1,1))
df['Trip_Count'] = y
test= df['Flight_Flag'].values #convert to numpy array
y = scaler.fit_transform(test.reshape(-1,1))
df['Flight_Flag'] = y
test= df['Medicare_Percent'].values #convert to numpy array
y = scaler.fit_transform(test.reshape(-1,1))
df['Medicare_Percent'] = y
test= df['Medicaid_Percent'].values #convert to numpy array
y = scaler.fit_transform(test.reshape(-1,1))
df['Medicaid_Percent'] = y
test= df['Insurance_Percent'].values #convert to numpy array
y = scaler.fit_transform(test.reshape(-1,1))
df['Insurance_Percent'] = y
test= df['Contract_Percent'].values #convert to numpy array
y = scaler.fit_transform(test.reshape(-1,1))
df['Contract_Percent'] = y
test= df['Bill_Patient_Percent'].values #convert to numpy array
y = scaler.fit_transform(test.reshape(-1,1))
df['Bill_Patient_Percent'] = y
test= df['Private_Pay_Percent'].values #convert to numpy array
y = scaler.fit_transform(test.reshape(-1,1))
df['Private_Pay_Percent'] = y
test= df['Emerg_Percent_Trips'].values #convert to numpy array
y = scaler.fit_transform(test.reshape(-1,1))
df['Emerg_Percent_Trips'] = y
test= df['Non_Emerg_Percent_Trip'].values #convert to numpy array
y = scaler.fit_transform(test.reshape(-1,1))
df['Non_Emerg_Percent_Trip'] = y
test= df['QMC_Revenue'].values #convert to numpy array
y = scaler.fit_transform(test.reshape(-1,1))
df['QMC_Revenue'] = y
test= df['TripCOGS'].values #convert to numpy array
y = scaler.fit_transform(test.reshape(-1,1))
df['TripCOGS'] = y
test= df['Avg_Rev_Per_Trip'].values #convert to numpy array
y = scaler.fit_transform(test.reshape(-1,1))
df['Avg_Rev_Per_Trip'] = y
test= df['Avg_Cogs_Per_Trip'].values #convert to numpy array
y = scaler.fit_transform(test.reshape(-1,1))
df['Avg_Cogs_Per_Trip'] = y
test= df['Avg_Profit_Per_Trip'].values #convert to numpy array
y = scaler.fit_transform(test.reshape(-1,1))
df['Avg_Profit_Per_Trip'] = y
test= df['Profit'].values #convert to numpy array
y = scaler.fit_transform(test.reshape(-1,1))
df['Profit'] = y
test= df['Market Type'].values #convert to numpy array
y = scaler.fit_transform(test.reshape(-1,1))
df['Market Type'] = y
test= df['Ground/Air'].values #convert to numpy array
y = scaler.fit_transform(test.reshape(-1,1))
df['Ground/Air'] = y

In [ ]:
df.head()

In [ ]:
from sklearn import metrics
from sklearn.cluster import KMeans

In [ ]:
wcss = []
for i in range(2, 20):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 42)
    kmeans.fit(df)
    wcss.append(kmeans.inertia_)

In [ ]:
plt.plot(range(2, 20), wcss,marker='o')
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters = 7, init = 'k-means++', random_state = 42)
kmeans.fit(df)
pred = kmeans.predict(df)

In [ ]:
qmc_data['cluster'] = pred

In [ ]:
def plot_stats(df_name,feature,label_rotation=False):
    temp = df_name[feature].value_counts()
    df1 = pd.DataFrame({feature: temp.index,'Count': temp.values})
    sbs.set_color_codes("pastel")
    s = sbs.barplot(x = feature, y="Count",data=df1)
    if(label_rotation):
        s.set_xticklabels(s.get_xticklabels(),rotation=90)
    plt.tick_params(axis='both', which='major', labelsize=10)
    plt.show();

In [ ]:
plot_stats(qmc_data,'cluster')

In [ ]:
qmc_data.to_csv(r'Cluster_File_0708.csv', index = False)

In [ ]:
fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(qmc_data['Avg_Monthly_Trips'][qmc_data.cluster == 0], qmc_data["Avg_Rev_Per_Trip"][qmc_data.cluster == 0], qmc_data["Avg_Profit_Per_Trip"][qmc_data.cluster == 0], c='blue', s=60, label='cluster 1')
ax.scatter(qmc_data['Avg_Monthly_Trips'][qmc_data.cluster == 1], qmc_data["Avg_Rev_Per_Trip"][qmc_data.cluster == 1], qmc_data["Avg_Profit_Per_Trip"][qmc_data.cluster == 1], c='red', s=60, label='cluster 2')
ax.scatter(qmc_data['Avg_Monthly_Trips'][qmc_data.cluster == 2], qmc_data["Avg_Rev_Per_Trip"][qmc_data.cluster == 2], qmc_data["Avg_Profit_Per_Trip"][qmc_data.cluster == 2], c='green', s=60, label='cluster 3')
ax.scatter(qmc_data['Avg_Monthly_Trips'][qmc_data.cluster == 3], qmc_data["Avg_Rev_Per_Trip"][qmc_data.cluster == 3], qmc_data["Avg_Profit_Per_Trip"][qmc_data.cluster == 3], c='purple', s=60, label='cluster 4')
ax.scatter(qmc_data['Avg_Monthly_Trips'][qmc_data.cluster == 4], qmc_data["Avg_Rev_Per_Trip"][qmc_data.cluster == 4], qmc_data["Avg_Profit_Per_Trip"][qmc_data.cluster == 4], c='pink', s=60, label='cluster 5')
ax.scatter(qmc_data['Avg_Monthly_Trips'][qmc_data.cluster == 5], qmc_data["Avg_Rev_Per_Trip"][qmc_data.cluster == 5], qmc_data["Avg_Profit_Per_Trip"][qmc_data.cluster == 5], c='gold', s=60, label='cluster 6')
ax.scatter(qmc_data['Avg_Monthly_Trips'][qmc_data.cluster == 6], qmc_data["Avg_Rev_Per_Trip"][qmc_data.cluster == 6], qmc_data["Avg_Profit_Per_Trip"][qmc_data.cluster == 6], c='black', s=60, label='cluster 7')

ax.ticklabel_format(style='plain')

ax.view_init(30, 185)
plt.xlabel("Avg_Monthly_Trip")
plt.ylabel("Avg_Rev_Per_Trip")
ax.set_zlabel('Avg_Profit_Per_Trip')
ax.legend()
plt.show()

In [ ]:
y = df[['Medicare_Percent','Medicaid_Percent','Insurance_Percent','Contract_Percent','Bill_Patient_Percent','Private_Pay_Percent','QMC_Revenue','TripCOGS','Profit']]

In [ ]:
wcss = []
for i in range(2, 20):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 42)
    kmeans.fit(y)
    wcss.append(kmeans.inertia_)

In [ ]:
plt.plot(range(2, 20), wcss,marker='o')
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters = 6, init = 'k-means++', random_state = 42)
kmeans.fit(y)
pred = kmeans.predict(y)

In [ ]:
qmc_data = qmc_data.drop(['cluster'], axis = 1)

In [ ]:
qmc_data['cluster'] = pred

In [ ]:
plot_stats(qmc_data,'cluster')

In [ ]:
fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(qmc_data['Avg_Monthly_Trips'][qmc_data.cluster == 0], qmc_data["Avg_Rev_Per_Trip"][qmc_data.cluster == 0], qmc_data["Avg_Profit_Per_Trip"][qmc_data.cluster == 0], c='blue', s=60, label='cluster 1')
ax.scatter(qmc_data['Avg_Monthly_Trips'][qmc_data.cluster == 1], qmc_data["Avg_Rev_Per_Trip"][qmc_data.cluster == 1], qmc_data["Avg_Profit_Per_Trip"][qmc_data.cluster == 1], c='red', s=60, label='cluster 2')
ax.scatter(qmc_data['Avg_Monthly_Trips'][qmc_data.cluster == 2], qmc_data["Avg_Rev_Per_Trip"][qmc_data.cluster == 2], qmc_data["Avg_Profit_Per_Trip"][qmc_data.cluster == 2], c='green', s=60, label='cluster 3')
ax.scatter(qmc_data['Avg_Monthly_Trips'][qmc_data.cluster == 3], qmc_data["Avg_Rev_Per_Trip"][qmc_data.cluster == 3], qmc_data["Avg_Profit_Per_Trip"][qmc_data.cluster == 3], c='purple', s=60, label='cluster 4')
ax.scatter(qmc_data['Avg_Monthly_Trips'][qmc_data.cluster == 4], qmc_data["Avg_Rev_Per_Trip"][qmc_data.cluster == 4], qmc_data["Avg_Profit_Per_Trip"][qmc_data.cluster == 4], c='pink', s=60, label='cluster 5')
ax.scatter(qmc_data['Avg_Monthly_Trips'][qmc_data.cluster == 5], qmc_data["Avg_Rev_Per_Trip"][qmc_data.cluster == 5], qmc_data["Avg_Profit_Per_Trip"][qmc_data.cluster == 5], c='gold', s=60, label='cluster 6')

ax.ticklabel_format(style='plain')

ax.view_init(30, 185)
plt.xlabel("Avg_Monthly_Trip")
plt.ylabel("Avg_Rev_Per_Trip")
ax.set_zlabel('Avg_Profit_Per_Trip')
ax.legend()
plt.show()

In [ ]:
qmc_data.to_csv(r'Cluster_File_2_0708.csv', index = False)

In [ ]:
sbs.scatterplot("Profit","TripCOGS", data=qmc_data, hue="cluster", palette="deep")

### Model 3

In [ ]:
x = df[['Avg_Monthly_Trips','Avg_Rev_Per_Trip','Avg_Cogs_Per_Trip','Profit','Market Type','Avg_Profit_Per_Trip']]

In [ ]:
wcss = []
for i in range(2, 20):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 42)
    kmeans.fit(x)
    wcss.append(kmeans.inertia_)

In [ ]:
plt.plot(range(2, 20), wcss,marker='o')
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters = 4, init = 'k-means++', random_state = 42)
kmeans.fit(x)
pred = kmeans.predict(x)

In [ ]:
qmc_data = qmc_data.drop(['cluster'], axis = 1)

In [ ]:
qmc_data['cluster'] = pred

In [ ]:
plot_stats(qmc_data,'cluster')

In [ ]:
sbs.scatterplot("Profit","TripCOGS", data=qmc_data, hue="cluster", palette="deep")

In [ ]:
fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(qmc_data['Avg_Monthly_Trips'][qmc_data.cluster == 0], qmc_data["Avg_Rev_Per_Trip"][qmc_data.cluster == 0], qmc_data["Avg_Profit_Per_Trip"][qmc_data.cluster == 0], c='blue', s=60, label='cluster 1')
ax.scatter(qmc_data['Avg_Monthly_Trips'][qmc_data.cluster == 1], qmc_data["Avg_Rev_Per_Trip"][qmc_data.cluster == 1], qmc_data["Avg_Profit_Per_Trip"][qmc_data.cluster == 1], c='red', s=60, label='cluster 2')
ax.scatter(qmc_data['Avg_Monthly_Trips'][qmc_data.cluster == 2], qmc_data["Avg_Rev_Per_Trip"][qmc_data.cluster == 2], qmc_data["Avg_Profit_Per_Trip"][qmc_data.cluster == 2], c='green', s=60, label='cluster 3')
ax.scatter(qmc_data['Avg_Monthly_Trips'][qmc_data.cluster == 3], qmc_data["Avg_Rev_Per_Trip"][qmc_data.cluster == 3], qmc_data["Avg_Profit_Per_Trip"][qmc_data.cluster == 3], c='purple', s=60, label='cluster 4')


ax.ticklabel_format(style='plain')

ax.view_init(30, 185)
plt.xlabel("Avg_Monthly_Trip")
plt.ylabel("Avg_Rev_Per_Trip")
ax.set_zlabel('Avg_Profit_Per_Trip')
ax.legend()
plt.show()

In [ ]:
sbs.scatterplot("Avg_Rev_Per_Trip","Avg_Cogs_Per_Trip", data=qmc_data, hue="cluster", palette="deep")

In [ ]:
qmc_data.to_csv(r'Cluster_File_3_0708.csv', index = False)

### Model 4

In [ ]:
x = df[['Geo','Avg_Monthly_Trips','Avg_Rev_Per_Trip','Avg_Cogs_Per_Trip','Market Type','Avg_Profit_Per_Trip']]

In [ ]:
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 42)
    kmeans.fit(x)
    wcss.append(kmeans.inertia_)

In [ ]:
plt.plot(range(1, 11), wcss,marker='o')
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters = 4, init = 'k-means++', random_state = 42)
kmeans.fit(x)
pred = kmeans.predict(x)

In [ ]:
qmc_data = qmc_data.drop(['cluster'], axis = 1)

In [ ]:
qmc_data['cluster'] = pred

In [ ]:
plot_stats(qmc_data,'cluster')

In [ ]:
fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(qmc_data['Avg_Monthly_Trips'][qmc_data.cluster == 0], qmc_data["Avg_Rev_Per_Trip"][qmc_data.cluster == 0], qmc_data["Avg_Profit_Per_Trip"][qmc_data.cluster == 0], c='blue', s=60, label='cluster 1')
ax.scatter(qmc_data['Avg_Monthly_Trips'][qmc_data.cluster == 1], qmc_data["Avg_Rev_Per_Trip"][qmc_data.cluster == 1], qmc_data["Avg_Profit_Per_Trip"][qmc_data.cluster == 1], c='red', s=60, label='cluster 2')
ax.scatter(qmc_data['Avg_Monthly_Trips'][qmc_data.cluster == 2], qmc_data["Avg_Rev_Per_Trip"][qmc_data.cluster == 2], qmc_data["Avg_Profit_Per_Trip"][qmc_data.cluster == 2], c='green', s=60, label='cluster 3')
ax.scatter(qmc_data['Avg_Monthly_Trips'][qmc_data.cluster == 3], qmc_data["Avg_Rev_Per_Trip"][qmc_data.cluster == 3], qmc_data["Avg_Profit_Per_Trip"][qmc_data.cluster == 3], c='purple', s=60, label='cluster 4')


ax.ticklabel_format(style='plain')

ax.view_init(30, 185)
plt.xlabel("Avg_Monthly_Trip")
plt.ylabel("Avg_Rev_Per_Trip")
ax.set_zlabel('Avg_Profit_Per_Trip')
ax.legend()
plt.show()

In [ ]:
sbs.scatterplot("Avg_Rev_Per_Trip","Avg_Cogs_Per_Trip", data=qmc_data, hue="cluster", palette="deep")

In [ ]:
qmc_data.to_csv(r'Cluster_File_4_0708.csv', index = False)

### Model 5

In [ ]:
x = df[['Geo','Avg_Monthly_Trips','Avg_Rev_Per_Trip','Avg_Cogs_Per_Trip','Market Type','Avg_Profit_Per_Trip','Ground/Air']]

In [ ]:
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 42)
    kmeans.fit(x)
    wcss.append(kmeans.inertia_)

In [ ]:
plt.plot(range(1, 11), wcss,marker='o')
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters = 5, init = 'k-means++', random_state = 42)
kmeans.fit(x)
pred = kmeans.predict(x)

In [ ]:
qmc_data = qmc_data.drop(['cluster'], axis = 1)

In [ ]:
qmc_data['cluster'] = pred

In [ ]:
plot_stats(qmc_data,'cluster')

In [ ]:
sbs.scatterplot("Avg_Rev_Per_Trip","Avg_Cogs_Per_Trip", data=qmc_data, hue="cluster", palette="deep")

In [ ]:
fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(qmc_data['Avg_Cogs_Per_Trip'][qmc_data.cluster == 0], qmc_data["Avg_Rev_Per_Trip"][qmc_data.cluster == 0], qmc_data["Avg_Profit_Per_Trip"][qmc_data.cluster == 0], c='blue', s=60, label='cluster 1')
ax.scatter(qmc_data['Avg_Cogs_Per_Trip'][qmc_data.cluster == 1], qmc_data["Avg_Rev_Per_Trip"][qmc_data.cluster == 1], qmc_data["Avg_Profit_Per_Trip"][qmc_data.cluster == 1], c='red', s=60, label='cluster 2')
ax.scatter(qmc_data['Avg_Cogs_Per_Trip'][qmc_data.cluster == 2], qmc_data["Avg_Rev_Per_Trip"][qmc_data.cluster == 2], qmc_data["Avg_Profit_Per_Trip"][qmc_data.cluster == 2], c='green', s=60, label='cluster 3')
ax.scatter(qmc_data['Avg_Cogs_Per_Trip'][qmc_data.cluster == 3], qmc_data["Avg_Rev_Per_Trip"][qmc_data.cluster == 3], qmc_data["Avg_Profit_Per_Trip"][qmc_data.cluster == 3], c='purple', s=60, label='cluster 4')
ax.scatter(qmc_data['Avg_Cogs_Per_Trip'][qmc_data.cluster == 4], qmc_data["Avg_Rev_Per_Trip"][qmc_data.cluster == 4], qmc_data["Avg_Profit_Per_Trip"][qmc_data.cluster == 4], c='black', s=60, label='cluster 4')


ax.ticklabel_format(style='plain')

ax.view_init(30, 185)
plt.xlabel("Avg_Cogs_Per_Trip")
plt.ylabel("Avg_Rev_Per_Trip")
ax.set_zlabel('Avg_Profit_Per_Trip')
ax.legend()
plt.show()

In [ ]:
qmc_data.to_csv(r'Cluster_File_5_0708.csv', index = False)